# 2017 US Demand

In [20]:
#run this to install the package
%pip install CFEDemands>=0.6.3.dev0
%pip install matplotlib
%pip install xarray
%pip install ConsumerDemands>= 0.4.3.dev0
%pip install eep153-tools

zsh:1: 0.6.3.dev0 not found
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement 0.4.3.dev0 (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for 0.4.3.dev0
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cfe

ImportError: cannot import name 'engel_curves' from 'demands' (/Users/marylaska/.venvs/vscode/lib/python3.13/site-packages/demands/__init__.py)

In [1]:
import pandas as pd
import numpy as np

In [3]:
x_orig = pd.read_csv("data/WWEIA - Food Expenditures 2017.csv")
x = x_orig.set_index(['i','t','m','j']).squeeze()
x.head()


grams  \
i     t    m   j                  
93704 2017 USA 11112210  122.00   
               11513600  248.00   
               11513700  248.00   
               14107030   28.35   
               24198739  100.00   

                                                                 food_name  
i     t    m   j                                                            
93704 2017 USA 11112210                                 Milk, low fat (1%)  
               11513600  Chocolate milk, made from syrup with low fat milk  
               11513700  Chocolate milk, made from syrup with fat free ...  
               14107030                      Cheese, mozzarella, part skim  
               24198739                     Chicken tenders or strips, nfs

In [4]:
d = pd.read_csv("data/WWEIA - Household Characteristics 2017.csv")
d.columns.name = 'k'
d = d.replace(np.nan,0)

d


k,i,t,m,age,sex,HH_size
0,93703,2017,USA,2.0,Female,5
1,93704,2017,USA,2.0,Male,4
2,93705,2017,USA,66.0,Female,1
3,93706,2017,USA,18.0,Male,5
4,93707,2017,USA,13.0,Male,7
...,...,...,...,...,...,...
9249,102952,2017,USA,70.0,Female,2
9250,102953,2017,USA,42.0,Male,1
9251,102954,2017,USA,41.0,Female,7
9252,102955,2017,USA,14.0,Female,4


In [5]:
# Expenditures x may have duplicate columns
x = x.groupby(['i','t','m','j', 'food_name']).sum()
x = x.replace(0,np.nan) # Replace zeros with missing

# Take logs of expenditures; call this y
y = np.log(x)
y.head()

grams
i     t    m   j        food_name                                                   
93704 2017 USA 11112210 Milk, low fat (1%)                                  4.804021
               11513600 Chocolate milk, made from syrup with low fat milk   5.513429
               11513700 Chocolate milk, made from syrup with fat free milk  5.513429
               14107030 Cheese, mozzarella, part skim                       3.344627
               24198739 Chicken tenders or strips, nfs                      4.605170

In [6]:
# Define age bins and labels
bins = [0, 4, 9, 14, 19, 31, 51, 100]
labels = ['00-03', '04-08', '09-13', '14-18', '19-30', '31-50', '51-99']

# Assign age group based on bins
d['age_group'] = pd.cut(d['age'], bins=bins, labels=labels, right=False)

# Create column name like "Females 00-03", "Males 14-18", etc.
d['group'] = d['sex'] + 's ' + d['age_group'].astype(str)

# Count each person as 1
d['count'] = 1

# Pivot the table to wide format: rows = (i, t, m), columns = group
df_grouped = d.pivot_table(index=['i', 't', 'm'], columns='group', values='count',  aggfunc='sum', fill_value=0)

# Flatten the column index
df_grouped.columns.name = None
df_grouped = df_grouped.reset_index()

# Add log household size
df_hsize = d.drop_duplicates(['i', 't', 'm'])[['i', 't', 'm', 'HH_size']]
df_hsize['log HSize'] = np.log(df_hsize['HH_size'])

# Merge back
final_d = pd.merge(df_grouped, df_hsize.drop(columns='HH_size'), on=['i', 't', 'm'])

final_d.set_index(['i','t','m'],inplace=True)
final_d.head()

,,,Females 00-03,Females 04-08,Females 09-13,Females 14-18,Females 19-30,Females 31-50,Females 51-99,Males 00-03,Males 04-08,Males 09-13,Males 14-18,Males 19-30,Males 31-50,Males 51-99,log HSize
i,t,m,,,,,,,,,,,,,,,
93703,2017,USA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.609438
93704,2017,USA,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.386294
93705,2017,USA,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.000000
93706,2017,USA,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1.609438
93707,2017,USA,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1.945910


In [21]:
import cfe

ImportError: cannot import name 'engel_curves' from 'demands' (/Users/marylaska/.venvs/vscode/lib/python3.13/site-packages/demands/__init__.py)

In [18]:

from cfe import Regression

result = Regression(y=y,d=final_d)

ImportError: cannot import name 'engel_curves' from 'demands' (/Users/marylaska/.venvs/vscode/lib/python3.13/site-packages/demands/__init__.py)